In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import datetime
import time
import numpy as np
from collections import Counter
import os
import copy
import joblib
import matplotlib.pyplot as plt

# Часть 2. Описательная часть методологии формирования обучающего датасета

#### Основные аспекты методики формирования датасета:
- учет влияния каждого компонента (т.е. тикера)
- аккумулирование в часовые таймфреймы параметров из минутных данных
- отказ от применения стандартных временных рядов как единственных входных данных в модель МО.


Что является ***общепринятым подходом*** технического анализа цены акции? ***Применение технических индикаторов***, налагаемых на временной ряд цены.

Излишне пояснять минус такого подхода. ***Основным минусом является зависимость показаний индикаторов исключительно от предыдущих колебаний цены единственной акции***. Эти колебания, в свою очередь, обусловлены множеством других факторов. Не последнюю, а может и самую важную роль, играет текущий индекс рынка. То есть инвесторы учитывают фазу рынка и корреляцию с другими инструментами рынка. То есть первая акция коррелирует со второй, вторая - с третьей, и так далее. Истинные причины (какая акция на какую влияет, и почему, и влияет ли вообще) остаются в большинстве случаев от нас скрытыми. То есть, опираясь на индикатор по одной акции, мы опираемся как бы на хвост, последний вагон поезда. Куда же поезд держит путь, какого размера состав поезда, какова политика локомотива - от нас остается скрытым. Цена акции не дает нам такого ответа.

В сложившейся ситуации многофакторного взаимного влияния можно избрать ***следующий подход, легко моделируемый математически и вполне понятный на обывательском уровне.***

Мы беремся управлять пониманием всего рынка в целом (точнее, 500 его основных компонентов). Чтобы ни одно движение ни одного из 500 компонентов не ускользнуло от нашего внимания, и мы смогли принимать более взвешенные и статистически подкрепленные решения. Если нам это удастся, ***мы сможем вместо инвестирования в "хвосты поезда", т.е. отдельные акции, инвестировать в ETF SP&500.***

На обывательском уровне идея проста и понятна. Давайте попробуем составить математический каркас одного из многочисленных способов реализации этой идеи.

(Не будем забывать следующее. Идей, которые можно выдвинуть на финансовых рынках - сотни, тысячи. Конкретных способов реализации этой идей - тоже тысячи. Итого миллионы возможных путей по которому мы могли пойти. С учетом этого, вопрос **"а почему выбрано именно такое действие? можно было по-другому" получает автоматический ответ "вообще-то есть еще миллионы вариантов действий. сейчас мы рассматриваем один вариант".**)

Итак, какие же конкретно шаги мы предпримем. Берем отдельный тикер, и поминутно в течение часа собираем по нему **количество признаков `highes`, свидетельствующих о желательности входа в позицию** Лонг (мы ждем роста рынка в ближайшие минуты). Отдельно собираем количество признаков **`lowes`** для позиции Шорт (мы ждем падения рынка в ближайшие минуты). И так по каждому тикеру. В итоге суммируем количество признаков отдельно для позиций Лонг и Шорт.

Таким образом, мы получаем для характеристики текущего часа два числа. Существенное превышение числа **`highes`** над числом **`lowes`** даст нам повод предположить, что рынок в ближайшие часы будет расти. И наоборот, существенное превышение **`lowes`** над **`highes`** даст повод думать, что рынок будет падать.

Ну и чтобы совсем не отказываться от такого фактора как цена, вводим третью характеристику **`market_index_rolling`** - изменение уровня/цены индекса S&P 500 текущего часа по сравнению со средним уровнем за последние 32 часа, допустим. Это позволит модели МО соотносить числа **`lowes`** и **`highes`** в каждом часе с тем, насколько все это влияет на уровень индекса в следующем часе. И то, что мы берем изменение индекса, а не сам индекс, позволяет нам добиться стационарности данных, что необходимо для модели МО.

Но мы же помним, что работаем с временными данными. При формировании датасета для модели МО, и тем более при применении нейронных LSTM-сетей или GRU-сетей нам нужно подавать на вход временной ряд. Поэтому нам из наших данных за текущий час (**`lowes`**, **`highes`**, **`market_index_rolling`**) нужно получить аналог временных данных. Как? Мы просто берем эти данные не за 1 час, а за те же последние 32 часа, допустим. В итоге получаем 3 * 32 = 96 итоговых признаков для формирования датасета.

Что же будет целевым столбцом - т.н. таргетом? В нашем случае таргетом будет и само изменение цены (то есть регрессионная задача МО), и отдельно продемонстрируем пример рекомендательной системы как решение задачи классификации с 3-мя классами (то есть покупать/продавать, или же удержание позиции "холд" - ничего не делать).

#### Важное примечание

Важно помнить, что такой параметр как - "поминутное количество признаков, свидетельствующих о желательности входа в позицию" - это абсолютно опциональная вещь. Это параметр может реализовать собственноручно написанная единственная функция в три строки, которая в корне поменяет результативность исследования.

Нижеприведенный код реализует описанную выше логику. В этой части мы формируем датасет с примером формирования таргета в виде классов. Обучение модели МО на основе датасета и анализ, интерпретация результатов показаны в следующем файле ноутбука.

# Часть 3. Функциональная часть кода формирования датасета

In [ ]:
# Данные: загрузка
def traindata_making_standard():
    path = './df_marketdata/'
    companies_list = []
    for root, dirs, files in os.walk(path):
        for _file in files:
            companies_list.append(_file)
    companies_list = sorted(companies_list)
    DF = {}
    for company in companies_list:
        try:
            df = joblib.load('./df_marketdata/'+company)
            DF[company] = df
        except:
            print(company)
            pass    
    
    return DF

#Данные: ресэмплирование на нужные минутные интервалы перед подсчетом точек
def resampling_standard_data(DF, minutes=5):
    
    resample_time=str(minutes)+str('Min')

    DF_1={}
    for i in DF.keys():
        try:

            df1=DF[i].copy(deep=True)
            df=pd.DataFrame()
            df['High'], df['Low']=df1['High'].resample(resample_time).max(), df1['Low'].resample(resample_time).min()
            df['company']=str(i)
            df=df[df['High'].notnull()]


            DF_1[i]=df

        except:
            pass
    return DF_1



In [ ]:
#ПОДСЧЕТ МОМЕНТОВ УВЕРЕННОГО РАЗВОРОТА ПО РЕСЭМПЛИРОВАННЫМ ТОЧКАМ - по одному тикеру
# Возвращается список таймстэмпов
def get_high_low_timestamps_for_one_df(df_newest):
    spisok_low=[]
    spisok_high=[]
    for y in range(10, len(df_newest)):
        try:
            if df_newest['Srlow'].iat[y] >= 0:
                y_point=y
                y=y-1
                while not df_newest['Srlow'].iat[y] >= 0:
                    y=y-1
                srlow3=y

                y=y-1
                while not df_newest['Srlow'].iat[y] >= 0:
                    y=y-1
                srlow2=y

                y=y_point-1                     
                while not df_newest['Srhigh'].iat[y] >= 0:
                    y=y-1
                srhigh=y                    

                if srhigh<srlow3:
                    if df_newest['Srlow'].iat[y_point]>df_newest['Srlow'].iat[srlow3] and df_newest['Srlow'].iat[srlow2]>df_newest['Srlow'].iat[srlow3]:
                        moment_low=df_newest.index[srhigh]
                        spisok_low.append(moment_low)
        except:
            pass
    # а теперь по моментам хай    
    for y in range(10, len(df_newest)):
        try:
            if df_newest['Srhigh'].iat[y] >= 0:
                y_point=y
                y=y-1
                while not df_newest['Srhigh'].iat[y] >= 0:
                    y=y-1
                srhigh3=y

                y=y-1
                while not df_newest['Srhigh'].iat[y] >= 0:
                    y=y-1
                srhigh2=y

                y=y_point-1                     
                while not df_newest['Srlow'].iat[y] >= 0:
                    y=y-1
                srlow=y                    

                if srlow<srhigh3:
                    if df_newest['Srhigh'].iat[y_point]<df_newest['Srhigh'].iat[srhigh3] and df_newest['Srhigh'].iat[srhigh2]<df_newest['Srhigh'].iat[srhigh3]:
                        moment_high=df_newest.index[srlow]
                        spisok_high.append(moment_high)
        except:
            pass
    return spisok_high, spisok_low

In [ ]:
#Получение полного списка таймстэмпов по всем тикерам
def get_timestamps_for_all(traindata_resampled):
    companies = traindata_resampled.keys()
    spisok_high_global = []
    spisok_low_global = []
    for company in companies:
        df=traindata_resampled[company].copy(deep=True)
        df_newest = get_fractal_points_for_one_df(df)
        spisok_high, spisok_low = get_high_low_timestamps_for_one_df(df_newest)
        spisok_high_global.append(spisok_high)
        spisok_low_global.append(spisok_low)
    spisok_high_all = []
    spisok_low_all = []    
    for item in spisok_high_global:
        for i in item:
            spisok_high_all.append(i)
    for item in spisok_low_global:
        for i in item:
            spisok_low_all.append(i)            
            
    
    return spisok_high_all, spisok_low_all

In [ ]:
#получение чернового датасета по минутам
def get_unsampled_dataset(traindata_standard, times = [3,7,15]):
    step = 0
    for minute in times:
        traindata_resampled = resampling_standard_data(traindata_standard, minutes=minute)
        spisok_high_all, spisok_low_all = get_timestamps_for_all(traindata_resampled)
        if step == 0:
            max_time = max(max(spisok_high_all), max(spisok_low_all))
            min_time = min(min(spisok_high_all), min(spisok_low_all))
            indexes=pd.date_range(start=min_time, end=max_time, freq='1Min')
            
            
        
            
            dfs = pd.DataFrame(index=indexes)
            dfs['day_number'] = dfs.index.date
            dti=pd.date_range(start=min_time, end=max_time, freq='B')
            dfs_filtered=(dfs.loc[dfs['day_number'].isin(pd.DataFrame(index=dti).index.date)])
            dfs_filtered_between_time=dfs_filtered.between_time(start_time='09:30', end_time='16:00')
            
            
            unsampled_dataset = pd.DataFrame(index=dfs_filtered_between_time.index)
        step = step + 1
            
            
            
            
            
            
            
        
        slovar_high = Counter(spisok_high_all)
        slovar_low = Counter(spisok_low_all)

        unsampled_dataset['highes'+str(minute)] = pd.Series(data=list(slovar_high.values()), index=list(slovar_high.keys()))
        unsampled_dataset['lowes'+str(minute)] = pd.Series(data=list(slovar_low.values()), index=list(slovar_low.keys()))

    return unsampled_dataset

In [ ]:
#Данные: после ресэмплирования, получение точек для одного тикера
def get_fractal_points_for_one_df(df):
    
    hh=(df['High']>df['Low'].rolling(3).mean())&(df['High']>df['Low'].shift(-1)*1.002)
    ll=(df['High']<df['Low'].rolling(3).mean()*1.01)&(df['High']<df['Low'].shift(-1)*1.05)

    df_new_hh=pd.DataFrame({'High':df[hh].High})
    frac_hh=(df_new_hh['High']>df_new_hh['High'].shift(-4))&(df_new_hh['High']>df_new_hh['High'].shift(-4))

    df_new_ll=pd.DataFrame({'Low':df[ll].Low})
    frac_ll=(df_new_ll['Low']<df_new_ll['Low'].rolling(10).mean()*1.01)&(df_new_ll['Low']<df_new_ll['Low'].shift(-1))

    df_newest=pd.DataFrame({'Srhigh':df_new_hh[frac_hh].High,'Srlow':df_new_ll[frac_ll].Low})
    
    return df_newest

In [ ]:
# вспомогательная функция. применяется в случае тренировки данных из разных источников
def traindata_standard_together(traindata_standard_previous, traindata_standard_current):
    traindata_standard={}
    for company in traindata_standard_previous.keys():
        try:
            df = traindata_standard_previous[company]
            traindata_standard[company]=df.append(traindata_standard_current[company])
        except:
            pass
    return traindata_standard

In [ ]:
#ресэмплирование к периоду 1 час всех периодов (3, 7, 15 минут) - методом суммы по каждому периоду
#оставляем только время от 09.00 до 16.00
def get_resampled_dataset(unsampled_dataset):
    resampled_dataset = unsampled_dataset.resample('H', label = 'right').sum()
    return resampled_dataset

In [ ]:
#формирование датасета из цен акций - для получения индекса рынка по часам
def get_dataset_for_market_index(traindata_standard):
    step = 0
    companies = traindata_standard.keys()
    DF={}
    for company in companies:
        df=traindata_standard[company].copy(deep=True)
        df['price'] = (df['High'] + df['Low']) / 2
        #df.drop(columns=['High', 'Low', 'company'], inplace=True)
        DF[company] = df['price']
    pddf=pd.DataFrame(DF)
    df=pddf.resample('H', label = 'right').mean()

    if step == 0:
        indexes=pd.date_range(start=df.index[0], end=df.index[-1], freq='H')
        dfs = pd.DataFrame(index=indexes)
        dfs['day_number'] = dfs.index.date
        dti=pd.date_range(start=df.index[0], end=df.index[-1], freq='B')
        dfs_filtered=(dfs.loc[dfs['day_number'].isin(pd.DataFrame(index=dti).index.date)])
        dfs_filtered_between_time=dfs_filtered.between_time(start_time='09:30', end_time='16:00')
    df_filtered = (df.index.isin(dfs_filtered_between_time.index))
    pddf=df[df_filtered]
    
    
    step = step + 1    
    
    dataset_for_market_index=pddf/pddf.shift(1)-1
    dataset_for_market_index=(1+dataset_for_market_index).cumprod()
    return dataset_for_market_index

In [ ]:
#получение индекса рынка по часам
def get_market_index_by_hour(dataset_for_market_index):
    return dataset_for_market_index.transpose().mean()

In [ ]:
#получение изменения среднего индекса рынка по часам (32 часа)
def get_market_index_rolling(market_index_by_hour):
    market_index_by_hour.dropna(inplace=True)
    market_index_rolling = market_index_by_hour - market_index_by_hour.rolling(32).mean()
    return market_index_rolling

In [ ]:
# справочная вспомогательная функция
def get_market_index_for_plotting(market_index_by_hour):
    market_index_by_hour.dropna(inplace=True)
    market_index_for_plotting = market_index_by_hour
    return market_index_for_plotting

In [ ]:
#получение датасета (с индексом рынка и только по рабочим бизнес-дням) для вычисления финального датасета
def get_dataset_for_calculation(resampled_dataset, market_index_rolling):
    resampled_dataset['market_index_rolling'] = market_index_rolling
    mask = (resampled_dataset['market_index_rolling'].notnull())
    dataset_for_calculation = resampled_dataset[mask]
    return dataset_for_calculation

In [6]:
#получаем имена столбцов датасета
def get_224__columnnames_for_dataset():
    columns_mask = ['highes3','lowes3','highes7','lowes7','highes15','lowes15','market_index_rolling']
    global_mask = []
    for i in range(32):
        mask = []
        for j in columns_mask:
            imya = str(i) + '_' + str(j)
            mask.append(imya)
        global_mask = global_mask + mask
    return global_mask

In [ ]:
#простираем вправо на 32 часа прежний датасет, с именами столбцов. датасет уже для маркировки
def get_ready_dataset_for_marking(dataset_for_calculation):
    calculated_dataset_for_marking = []
    dataset_for_marking_indexes = []
    for row in range(31, len(dataset_for_calculation)):
        row_list=[]
        for item in range(32):
            row_list = row_list + list(dataset_for_calculation.iloc[row - item].values)
        calculated_dataset_for_marking.append(row_list)
        dataset_for_marking_indexes.append(dataset_for_calculation.index[row])
    global_mask = get_224__columnnames_for_dataset()
    ready_dataset_for_marking = pd.DataFrame(data=calculated_dataset_for_marking, index=dataset_for_marking_indexes, columns=global_mask)
    return ready_dataset_for_marking

In [ ]:
# получаем первоначальный таргет-столбец с 4-мя классами. служит для иллюстрации
# способов составления таргет-столбца. в дальнейшем будут более
# релевантные поведению инвестора функции для таргет-столбцов.
# столбец отражает, что случится раньше:
    # - падение рынка более чем на 1% и дальнейший рост более чем на 4%
    # - падение рынка более чем на 1% и дальнейшее падение более чем на 2%
    # - рост рынка более чем на 1% и дальнейший рост более чем на 4%
    # - рост рынка более чем на 1% и дальнейшее падение более чем на 2%
def get_df_rates_1percent(market_index_by_hour):
    df = pd.DataFrame(market_index_by_hour)
    df['rates']=''
    df_list = list(market_index_by_hour)
    for i in range(len(df_list)):
        for k in range(i+1, len(df_list)):
            delta = df_list[k] - df_list[i]
            if delta/df_list[i] >0.01:
                df['rates'].iloc[i]=1
                break
            if delta/df_list[i] <-0.01:
                df['rates'].iloc[i]=-1
                break
    df = df[df['rates']!='']
    
    df['rates_add']=''
    df_list = list(market_index_by_hour)
    for i in range(len(df_list)):
        for k in range(i+1, len(df_list)):
            delta = df_list[k] - df_list[i]
            if delta/df_list[i] >0.04:
                df['rates_add'].iloc[i]=2
                break
            if delta/df_list[i] <-0.02:
                df['rates_add'].iloc[i]=-2
                break
    df = df[df['rates_add']!='']
    
    df['total_rates'] = df['rates'] + df['rates_add']
    
    
    
    return df['total_rates']

In [ ]:
#получение столбца для добавления - индекс рынка по часам
def get_market_index_by_hour_updated():
    df = joblib.load('SP.pkl')
    return df.Price
def get_market_index_rolling(market_index_by_hour):
    market_index_by_hour.dropna(inplace=True)
    market_index_rolling = market_index_by_hour - market_index_by_hour.rolling(32).mean()
    return market_index_rolling

In [ ]:
# формирование датасета с именами столбцов, и с таргетом
def percent_get_ready_dataset(dataset_for_calculation):
    calculated_dataset_for_marking = []
    dataset_for_marking_indexes = []
    for row in range(31, len(dataset_for_calculation)):
        row_list=[]
        for item in range(32):
            row_list = row_list + list(dataset_for_calculation.iloc[row - item].values)
        calculated_dataset_for_marking.append(row_list)
        dataset_for_marking_indexes.append(dataset_for_calculation.index[row])
    global_mask = get_224__columnnames_for_dataset()
    temp = pd.DataFrame(data=calculated_dataset_for_marking, index=dataset_for_marking_indexes, columns=global_mask)
    temp['class'] = get_df_rates_1percent(market_index_by_hour)
    percent_ready_dataset = temp[temp['class'].notnull()]
    return percent_ready_dataset

# Часть 4. Исполняемая часть кода. Запуск функций для формирования датасета

In [ ]:
traindata_standard = traindata_making_standard()
unsampled_dataset = get_unsampled_dataset(traindata_standard, times = [3,7,15])
resampled_dataset = get_resampled_dataset(unsampled_dataset)
dataset_for_market_index = get_dataset_for_market_index(traindata_standard)

In [ ]:
market_index_by_hour=get_market_index_by_hour_updated()
market_index_rolling = get_market_index_rolling(market_index_by_hour)
dataset_for_calculation = get_dataset_for_calculation(resampled_dataset, market_index_rolling)

In [ ]:
dataset = percent_get_ready_dataset(dataset_for_calculation)

In [ ]:
dataset['class'].value_counts()

# Часть 5. Получаем таргетированный столбец в виде регрессионной задачи

Многочисленные опыты с обучением полученного датасета не привели к адекватной результативности на тестовых данных. Причина видится в логике таргет-столбца. Таргет не учитывает, что цена в течение ближайших часов может вырасти как на 1%, так и на 3%. Разница колоссальная, а маркировка (то есть метка класса) одинаковая. Большее количество классов для учета всех ситуаций ведет к усложнению обучения модели МО. Поэтому рискнем и предположим таргет-столбец в виде самой цены. Точнее, разницы между `High` последующего часа и средней ценой `(High+Low)/2` текущего часа. Ниже пример вычисления:

   
High         | Low         | Target_class
:------------|:------------|:------------
     15      |     11      |  5 = 18-(15+11)/2
     18      |     12      |  

Настало время загрузить вспомогательный файл `'SP.pkl'`. Это SP&500 по часам.
Столбец `SP['Price']` в нем как раз и является вычисляемым как `(High+Low)/2`:

In [3]:
SP = joblib.load('SP.pkl')
SP

,Price,Price_high,Price_low
Date_time,,,
2008-01-02 10:00:00,1469.825,1471.77,1467.88
2008-01-02 11:00:00,1461.170,1465.26,1457.08
2008-01-02 12:00:00,1453.490,1458.26,1448.72
2008-01-02 13:00:00,1449.015,1452.37,1445.66
2008-01-02 14:00:00,1447.170,1450.37,1443.97
...,...,...,...
2019-07-26 12:00:00,3020.180,3023.46,3016.90
2019-07-26 13:00:00,3022.855,3023.93,3021.78
2019-07-26 14:00:00,3023.375,3024.52,3022.23


Итак, приходим к задаче регрессии.
Нижеприведенная функция получения таргет-столбца реализует эту логику:

In [ ]:
def get_regression_with_high_average_delta(dataset):
    SP = joblib.load('SP.pkl') # данный файл подготовлен на основе данных
                               # finam.ru
    temp = pd.DataFrame(index=dataset.index)
    temp['Price_high'] = SP['Price_high']
    temp['Price'] = SP['Price'] # это и есть цена (High+Low)/2
    temp['class'] = temp.Price_high.shift(-1) - temp.Price
    return temp['class']

In [ ]:
dataset['class'] = get_regression_with_high_average_delta(dataset)

In [ ]:
dataset = dataset[:-1] #избавляемся от последней строки с Nan

Тут конечно надо дать пояснения. Мы сделали очень серьезное допущение, приемлемое для целей машинного обучения. Но на этапе продакшна это может вызвать серьезную озабоченность.

Итак, речь идет о формуле: **таргет-столбец равен разнице между `High` последующего часа и средней ценой `(High+Low)/2` текущего часа**.

На практике имело бы смысл для текущего часа брать цену **`Close`**, так как именно от нее будет считаться финансовый результат. Именно по этой цене мы имеем все шансы войти в сделку. Насколько **`(High+Low)/2`** помешает нашим целям?

Предварительный ответ: не помешает. По статистическим причинам:
- В среднем, на протяжении длительного периода времени, цена **`Close`** практически совпадает с условной ценой **`(High+Low)/2`**. Матожидание отклонений составляет минус 0,1 пункт. Стандартное отклонение - 3 пункта. И это при уровне SP&500 2000-3000 пунктов. То есть, при рекомендациях модели, в какие-то моменты мы будем недобирать пару пунктов, в другие моменты - забирать себе эти лишние пункты. В среднем будем выходить на планируемые уровни прибыли.
- На бэктесте этот момент надо проверять, в любом случае. Вдруг модель будет выбирать в основном такие моменты, где мы будем терять на этих отклонениях.

Но зачем мы применили именно **`(High+Low)/2`** вместо привычного **`Close`**? Все по тем же причинам - общая концепция моделирования датасета. Датасет весь построен на отклонениях **`High и Low`**. Дело в том, что эти два уровня цен характеризуют весь временной период (минута, час, день). Показывают, насколько высоко ушел или низко упал рынок. То есть содержат сжатую информацию обо всем периоде. Между тем **`Close`** характеризует только один момент времени. **`Close`** статичен по своей природе, в отличие от динамичных **`High и Low`**.

При наличии цены **`Close`** мы получим только одну информацию на конец часа. Как закрылись торги в этом часе. А что происходило внутри часа, какие баталии, насколько рынок уходил вверх или падал вниз - мы не узнаем. Точнее даже не мы, а модель МО, для которой подобная информация (**`High и Low`**) может послужить ценным источником.

Таким образом, теряя (возможно) пару пунктов из-за принятого допущения, мы приобретаем улучшенную точность модели. И любые расчеты показывают, что **повышение точности модели перекроет любые условные допущения и связанные с ними потери**. Это прагматичный расчет, ничего личного.

# Часть 6. Сокращение размерности датасета, и сохранение окончательного датасета для машинного обучения

Вышеприведенный код реализовал подсчет минут на основе 3-х, 7-и, 15-и минутных интервалов. Практика показала, что без существенных потерь можно снизить размерность датасета до 96 признаков, опираясь только на 3-х минутные интервалы:

In [24]:
def delete_columns(dataset):
    for i in dataset.columns[:-1]:
        if ('s7' in i) or ('s15' in i):
            dataset.drop(columns=[i], inplace=True)
        else:
            pass
    return dataset

In [22]:
dataset = delete_columns(dataset)

In [75]:
joblib.dump(dataset, 'marked_dataset_for_regression_with_high_average_delta_.pkl')